In [12]:
import tensorflow as tf
from pathlib import Path

IMAGES_PATH = Path() / "images" / "deep"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

dense = tf.keras.layers.Dense(50, activation="relu", kernel_initializer="he_normal")

In [2]:
he_avg_init = tf.keras.initializers.VarianceScaling(scale=2., mode="fan_avg",
                                                    distribution="uniform")
dense = tf.keras.layers.Dense(50, activation="sigmoid",
                              kernel_initializer=he_avg_init)


In [3]:
# Leaky ReLU is better than regular ReLU
leaky_relu = tf.keras.layers.LeakyReLU(alpha=0.2)  # defaults to alpha=0.3
dense = tf.keras.layers.Dense(50, activation=leaky_relu,
                              kernel_initializer="he_normal")


In [4]:
# Batch normalization in keras
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense_3 (Dense)             (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_4 (Dense)             (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)              4

In [6]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])


In [8]:
# Gradient clipping
optimizer = tf.keras.optimizers.SGD(clipvalue=1.0)
model.compile(optimizer=optimizer)


In [12]:
# extra code – split Fashion MNIST into tasks A and B, then train and save
#              model A to "my_model_A".
import numpy as np
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]
X_train, X_valid, X_test = X_train / 255, X_valid / 255, X_test / 255
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]
pos_class_id = class_names.index("Pullover")
neg_class_id = class_names.index("T-shirt/top")

def split_dataset(X, y):
    y_for_B = (y == pos_class_id) | (y == neg_class_id)
    y_A = y[~y_for_B]
    y_B = (y[y_for_B] == pos_class_id).astype(np.float32)
    old_class_ids = list(set(range(10)) - set([neg_class_id, pos_class_id]))
    for old_class_id, new_class_id in zip(old_class_ids, range(8)):
        y_A[y_A == old_class_id] = new_class_id  # reorder class ids for A
    return ((X[~y_for_B], y_A), (X[y_for_B], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

tf.random.set_seed(42)

model_A = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(8, activation="softmax")
])

model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
                metrics=["accuracy"])
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                      validation_data=(X_valid_A, y_valid_A))
model_A.save("my_model_A")

Epoch 1/20
1376/1376 [==============================] - 2s 1ms/step - loss: 1.2435 - accuracy: 0.6356 - val_loss: 0.7593 - val_accuracy: 0.7659
Epoch 2/20
1376/1376 [==============================] - 2s 1ms/step - loss: 0.6516 - accuracy: 0.7922 - val_loss: 0.5509 - val_accuracy: 0.8235
Epoch 3/20
1376/1376 [==============================] - 2s 1ms/step - loss: 0.5242 - accuracy: 0.8328 - val_loss: 0.4721 - val_accuracy: 0.8403
Epoch 4/20
1376/1376 [==============================] - 2s 2ms/step - loss: 0.4592 - accuracy: 0.8513 - val_loss: 0.4221 - val_accuracy: 0.8581
Epoch 5/20
1376/1376 [==============================] - 2s 2ms/step - loss: 0.4165 - accuracy: 0.8639 - val_loss: 0.3889 - val_accuracy: 0.8669
Epoch 6/20
1376/1376 [==============================] - 2s 2ms/step - loss: 0.3862 - accuracy: 0.8730 - val_loss: 0.3663 - val_accuracy: 0.8744
Epoch 7/20
1376/1376 [==============================] - 2s 2ms/step - loss: 0.3642 - accuracy: 0.8784 - val_loss: 0.3489 - val_accuracy:

INFO:tensorflow:Assets written to: my_model_A/assets


INFO:tensorflow:Assets written to: my_model_A/assets


In [13]:
model_A = tf.keras.models.load_model("my_model_A")
model_B_on_A = tf.keras.Sequential(model_A.layers[:-1])
model_B_on_A.add(tf.keras.layers.Dense(1, activation="sigmoid"))
model_A_clone = tf.keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [14]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer,
                     metrics=["accuracy"])

In [15]:
# Train transfer model
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer,
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 1s 29ms/step - loss: 1.8742 - accuracy: 0.4450 - val_loss: 1.2454 - val_accuracy: 0.4847
Epoch 2/4
7/7 [==============================] - 0s 9ms/step - loss: 1.1230 - accuracy: 0.4500 - val_loss: 0.8472 - val_accuracy: 0.4916
Epoch 3/4
7/7 [==============================] - 0s 9ms/step - loss: 0.8492 - accuracy: 0.4350 - val_loss: 0.7699 - val_accuracy: 0.3610
Epoch 4/4
7/7 [==============================] - 0s 10ms/step - loss: 0.7862 - accuracy: 0.3350 - val_loss: 0.7550 - val_accuracy: 0.3927
Epoch 1/16
7/7 [==============================] - 1s 30ms/step - loss: 0.7471 - accuracy: 0.4050 - val_loss: 0.6897 - val_accuracy: 0.5687
Epoch 2/16
7/7 [==============================] - 0s 9ms/step - loss: 0.6806 - accuracy: 0.5600 - val_loss: 0.6281 - val_accuracy: 0.6963
Epoch 3/16
7/7 [==============================] - 0s 9ms/step - loss: 0.6196 - accuracy: 0.7150 - val_loss: 0.5943 - val_accuracy: 0.6884
Epoch 4/16
7/7 [===================

In [16]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 1ms/step - loss: 0.3430 - accuracy: 0.9210


[0.3430161476135254, 0.9210000038146973]

In [17]:
# implementing momentum optimization
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)

In [18]:
# Nesterov Accelerated Gradient (faster than momentum)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)

In [19]:
# RMS Prop, uses exp decay on most recent gradients
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)

In [20]:
# Adam, combination of momentum and RMS prop
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

In [22]:
# Changing learning rates
def exponential_decay_fn(epoch):
    return 0.01 * 0.1 ** (epoch/20)

In [23]:
# Regularization
layer = tf.keras.layers.Dense(100, activation="relu",
                              kernel_initializer="he_normal",
                              kernel_regularizer=tf.keras.regularizers.l2(0.01))

In [24]:
# Reuse regularization in several layers
from functools import partial

RegularizedDense = partial(tf.keras.layers.Dense,
                           activation="relu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=tf.keras.regularizers.l2(0.01))

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(100),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax")
])

In [32]:
# Dropout
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(10, activation="softmax")
])

# extra code – compile and train the model
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6943 - accuracy: 0.7439 - val_loss: 0.4625 - val_accuracy: 0.8268
Epoch 2/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.5201 - accuracy: 0.8098 - val_loss: 0.4039 - val_accuracy: 0.8498
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4799 - accuracy: 0.8244 - val_loss: 0.4071 - val_accuracy: 0.8476
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4534 - accuracy: 0.8337 - val_loss: 0.3852 - val_accuracy: 0.8552
Epoch 5/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.4405 - accuracy: 0.8380 - val_loss: 0.3698 - val_accuracy: 0.8590
Epoch 6/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.4276 - accuracy: 0.8432 - val_loss: 0.3795 - val_accuracy: 0.8552
Epoch 7/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.4194 - accuracy: 0.8455 - val_loss: 0.3581 - val_accuracy:

In [33]:
# Monte Carlo Dropout
import numpy as np

y_probas = np.stack([model(X_test, training=True)
                     for sample in range(100)])
y_proba = y_probas.mean(axis=0)

In [34]:
model.predict(X_test[:1]).round(3)

1/1 [==============================] - 0s 53ms/step


array([[0.   , 0.   , 0.   , 0.   , 0.   , 0.009, 0.   , 0.309, 0.   ,
        0.681]], dtype=float32)

In [35]:
y_proba[0].round(3)

array([0.001, 0.   , 0.   , 0.001, 0.   , 0.027, 0.001, 0.347, 0.002,
       0.622], dtype=float32)

In [36]:
y_std = y_probas.std(axis=0)
y_std[0].round(3)

array([0.002, 0.001, 0.001, 0.003, 0.001, 0.041, 0.003, 0.207, 0.008,
       0.213], dtype=float32)

In [37]:
y_pred = y_proba.argmax(axis=1)
accuracy = (y_pred == y_test).sum() / len(y_test)
accuracy

0.8659

In [38]:
class MCDropout(tf.keras.layers.Dropout):
    def call(self, inputs, training=False):
        return super().call(inputs, training=True)

In [39]:
# Max-norm regularization
dense = tf.keras.layers.Dense(
    100, activation="relu", kernel_initializer="he_normal",
    kernel_constraint=tf.keras.constraints.max_norm(1.))

### CIFAR10 dataset neural network

In [4]:
tf.random.set_seed(42)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(tf.keras.layers.Dense(100,
                                    activation="swish",
                                    kernel_initializer="he_normal"))

In [5]:
model.add(tf.keras.layers.Dense(10, activation="softmax"))

In [8]:
optimizer = tf.keras.optimizers.Nadam(learning_rate=5e-5)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [9]:
cifar10 = tf.keras.datasets.cifar10.load_data()
(X_train_full, y_train_full), (X_test, y_test) = cifar10

X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

In [13]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=20,
                                                     restore_best_weights=True)
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("my_cifar10_model",
                                                         save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = Path() / "my_cifar10_logs" / f"run_{run_index:03d}"
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [14]:
model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

Epoch 1/100
1399/1407 [============================>.] - ETA: 0s - loss: 3.3686 - accuracy: 0.1577

INFO:tensorflow:Assets written to: my_cifar10_model/assets


INFO:tensorflow:Assets written to: my_cifar10_model/assets


1407/1407 [==============================] - 19s 10ms/step - loss: 3.3617 - accuracy: 0.1581 - val_loss: 2.1264 - val_accuracy: 0.2120
Epoch 2/100
1407/1407 [==============================] - ETA: 0s - loss: 2.0629 - accuracy: 0.2387

INFO:tensorflow:Assets written to: my_cifar10_model/assets


...

INFO:tensorflow:Assets written to: my_cifar10_model/assets


1407/1407 [==============================] - 19s 13ms/step - loss: 1.3589 - accuracy: 0.5133 - val_loss: 1.5055 - val_accuracy: 0.4710
Epoch 30/100
1407/1407 [==============================] - 13s 9ms/step - loss: 1.3476 - accuracy: 0.5155 - val_loss: 1.5884 - val_accuracy: 0.4512
Epoch 31/100
1407/1407 [==============================] - 12s 9ms/step - loss: 1.3379 - accuracy: 0.5197 - val_loss: 1.5141 - val_accuracy: 0.4750
Epoch 32/100
1407/1407 [==============================] - 14s 10ms/step - loss: 1.3268 - accuracy: 0.5257 - val_loss: 1.5140 - val_accuracy: 0.4624
Epoch 33/100
1407/1407 [==============================] - 11s 8ms/step - loss: 1.3199 - accuracy: 0.5260 - val_loss: 1.5758 - val_accuracy: 0.4600
Epoch 34/100
1406/1407 [============================>.] - ETA: 0s - loss: 1.3128 - accuracy: 0.5310

INFO:tensorflow:Assets written to: my_cifar10_model/assets


INFO:tensorflow:Assets written to: my_cifar10_model/assets


1407/1407 [==============================] - 19s 14ms/step - loss: 1.3127 - accuracy: 0.5310 - val_loss: 1.4998 - val_accuracy: 0.4784
Epoch 35/100
1407/1407 [==============================] - 16s 11ms/step - loss: 1.3021 - accuracy: 0.5337 - val_loss: 1.5082 - val_accuracy: 0.4836
Epoch 36/100
1407/1407 [==============================] - 16s 11ms/step - loss: 1.2998 - accuracy: 0.5310 - val_loss: 1.5042 - val_accuracy: 0.4724
Epoch 37/100
1407/1407 [==============================] - 16s 11ms/step - loss: 1.2850 - accuracy: 0.5386 - val_loss: 1.5516 - val_accuracy: 0.4772
Epoch 38/100
1407/1407 [==============================] - 14s 10ms/step - loss: 1.2769 - accuracy: 0.5431 - val_loss: 1.5383 - val_accuracy: 0.4684
Epoch 39/100
1407/1407 [==============================] - 13s 9ms/step - loss: 1.2713 - accuracy: 0.5435 - val_loss: 1.5425 - val_accuracy: 0.4602
Epoch 40/100
1407/1407 [==============================] - 14s 10ms/step - loss: 1.2616 - accuracy: 0.5475 - val_loss: 1.5973 -

In [15]:
model.evaluate(X_valid, y_valid)

157/157 [==============================] - 0s 3ms/step - loss: 1.4998 - accuracy: 0.4784


[1.4998446702957153, 0.47839999198913574]

### With Batch Normalization

In [16]:
tf.random.set_seed(42)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(tf.keras.layers.Dense(100, kernel_initializer="he_normal"))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation("swish"))

model.add(tf.keras.layers.Dense(10, activation="softmax"))

optimizer = tf.keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=20,
                                                     restore_best_weights=True)
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("my_cifar10_bn_model",
                                                         save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = Path() / "my_cifar10_logs" / f"run_bn_{run_index:03d}"
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

model.evaluate(X_valid, y_valid)

Epoch 1/100
1404/1407 [============================>.] - ETA: 0s - loss: 2.0302 - accuracy: 0.2571

INFO:tensorflow:Assets written to: my_cifar10_bn_model/assets


INFO:tensorflow:Assets written to: my_cifar10_bn_model/assets


1407/1407 [==============================] - 41s 20ms/step - loss: 2.0301 - accuracy: 0.2572 - val_loss: 1.8664 - val_accuracy: 0.3328
Epoch 2/100
1407/1407 [==============================] - 17s 12ms/step - loss: 1.7720 - accuracy: 0.3636 - val_loss: 1.9310 - val_accuracy: 0.2996
Epoch 3/100
1404/1407 [============================>.] - ETA: 0s - loss: 1.6711 - accuracy: 0.4006

INFO:tensorflow:Assets written to: my_cifar10_bn_model/assets


...

1407/1407 [==============================] - 28s 20ms/step - loss: 1.3278 - accuracy: 0.5320 - val_loss: 1.4935 - val_accuracy: 0.4804
Epoch 12/100
1407/1407 [==============================] - 18s 13ms/step - loss: 1.2974 - accuracy: 0.5426 - val_loss: 1.5736 - val_accuracy: 0.4574
Epoch 13/100
1407/1407 [==============================] - 20s 14ms/step - loss: 1.2733 - accuracy: 0.5513 - val_loss: 1.5425 - val_accuracy: 0.4580
Epoch 14/100
1407/1407 [==============================] - 17s 12ms/step - loss: 1.2509 - accuracy: 0.5572 - val_loss: 1.6075 - val_accuracy: 0.4510
Epoch 15/100
1407/1407 [==============================] - 17s 12ms/step - loss: 1.2258 - accuracy: 0.5668 - val_loss: 1.6007 - val_accuracy: 0.4748
Epoch 16/100
1407/1407 [==============================] - ETA: 0s - loss: 1.2103 - accuracy: 0.5739

INFO:tensorflow:Assets written to: my_cifar10_bn_model/assets


INFO:tensorflow:Assets written to: my_cifar10_bn_model/assets


1407/1407 [==============================] - 29s 21ms/step - loss: 1.2103 - accuracy: 0.5739 - val_loss: 1.4523 - val_accuracy: 0.4964
Epoch 17/100
1407/1407 [==============================] - 23s 16ms/step - loss: 1.1879 - accuracy: 0.5803 - val_loss: 1.5757 - val_accuracy: 0.4688
Epoch 18/100
1407/1407 [==============================] - 21s 15ms/step - loss: 1.1681 - accuracy: 0.5884 - val_loss: 1.5822 - val_accuracy: 0.4472
Epoch 19/100
1407/1407 [==============================] - 19s 13ms/step - loss: 1.1543 - accuracy: 0.5924 - val_loss: 1.6824 - val_accuracy: 0.4230
Epoch 20/100
1407/1407 [==============================] - 15s 11ms/step - loss: 1.1333 - accuracy: 0.6015 - val_loss: 1.5253 - val_accuracy: 0.4728
Epoch 21/100
1407/1407 [==============================] - 16s 11ms/step - loss: 1.1225 - accuracy: 0.6046 - val_loss: 1.5253 - val_accuracy: 0.4798
Epoch 22/100
1407/1407 [==============================] - 20s 14ms/step - loss: 1.0997 - accuracy: 0.6145 - val_loss: 1.5130 

[1.452317237854004, 0.49639999866485596]

### Use SELU instead of batch normalization

In [17]:
tf.random.set_seed(42)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(tf.keras.layers.Dense(100,
                                    kernel_initializer="lecun_normal",
                                    activation="selu"))

model.add(tf.keras.layers.Dense(10, activation="softmax"))

optimizer = tf.keras.optimizers.Nadam(learning_rate=7e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    patience=20, restore_best_weights=True)
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    "my_cifar10_selu_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = Path() / "my_cifar10_logs" / f"run_selu_{run_index:03d}"
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=callbacks)

model.evaluate(X_valid_scaled, y_valid)

Epoch 1/100
1405/1407 [============================>.] - ETA: 0s - loss: 1.9229 - accuracy: 0.3115

INFO:tensorflow:Assets written to: my_cifar10_selu_model/assets


INFO:tensorflow:Assets written to: my_cifar10_selu_model/assets


1407/1407 [==============================] - 25s 14ms/step - loss: 1.9227 - accuracy: 0.3115 - val_loss: 1.7707 - val_accuracy: 0.3682
Epoch 2/100
1406/1407 [============================>.] - ETA: 0s - loss: 1.7122 - accuracy: 0.3910

INFO:tensorflow:Assets written to: my_cifar10_selu_model/assets


...

1407/1407 [==============================] - 19s 13ms/step - loss: 1.1925 - accuracy: 0.5850 - val_loss: 1.4619 - val_accuracy: 0.4986
Epoch 15/100
1407/1407 [==============================] - 13s 9ms/step - loss: 1.1632 - accuracy: 0.5981 - val_loss: 1.4968 - val_accuracy: 0.4972
Epoch 16/100
1407/1407 [==============================] - ETA: 0s - loss: 1.1518 - accuracy: 0.6016

INFO:tensorflow:Assets written to: my_cifar10_selu_model/assets


INFO:tensorflow:Assets written to: my_cifar10_selu_model/assets


1407/1407 [==============================] - 18s 13ms/step - loss: 1.1518 - accuracy: 0.6016 - val_loss: 1.4580 - val_accuracy: 0.5144
Epoch 17/100
1407/1407 [==============================] - 14s 10ms/step - loss: 1.1340 - accuracy: 0.6106 - val_loss: 1.5170 - val_accuracy: 0.5016
Epoch 18/100
1407/1407 [==============================] - 15s 11ms/step - loss: 1.1264 - accuracy: 0.6145 - val_loss: 1.5259 - val_accuracy: 0.5130
Epoch 19/100
1407/1407 [==============================] - 17s 12ms/step - loss: 1.1215 - accuracy: 0.6145 - val_loss: 1.5371 - val_accuracy: 0.5092
Epoch 20/100
1407/1407 [==============================] - 13s 10ms/step - loss: 1.0931 - accuracy: 0.6270 - val_loss: 1.5392 - val_accuracy: 0.4988
Epoch 21/100
1407/1407 [==============================] - 16s 11ms/step - loss: 1.0809 - accuracy: 0.6307 - val_loss: 1.5477 - val_accuracy: 0.5026
Epoch 22/100
1407/1407 [==============================] - 11s 8ms/step - loss: 1.0626 - accuracy: 0.6360 - val_loss: 1.5989 -

[1.45803701877594, 0.5144000053405762]

### Retrain Model with 1cycle

In [19]:
tf.random.set_seed(42)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(tf.keras.layers.Dense(100,
                                 kernel_initializer="lecun_normal",
                                 activation="selu"))

model.add(tf.keras.layers.AlphaDropout(rate=0.1))
model.add(tf.keras.layers.Dense(10, activation="softmax"))

optimizer = tf.keras.optimizers.SGD(learning_rate=2e-2)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [23]:
import math

K = tf.keras.backend

class OneCycleScheduler(tf.keras.callbacks.Callback):
    def __init__(self, iterations, max_lr=1e-3, start_lr=None,
                 last_iterations=None, last_lr=None):
        self.iterations = iterations
        self.max_lr = max_lr
        self.start_lr = start_lr or max_lr / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_lr = last_lr or self.start_lr / 1000
        self.iteration = 0

    def _interpolate(self, iter1, iter2, lr1, lr2):
        return (lr2 - lr1) * (self.iteration - iter1) / (iter2 - iter1) + lr1

    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            lr = self._interpolate(0, self.half_iteration, self.start_lr,
                                   self.max_lr)
        elif self.iteration < 2 * self.half_iteration:
            lr = self._interpolate(self.half_iteration, 2 * self.half_iteration,
                                   self.max_lr, self.start_lr)
        else:
            lr = self._interpolate(2 * self.half_iteration, self.iterations,
                                   self.start_lr, self.last_lr)
        self.iteration += 1
        K.set_value(self.model.optimizer.learning_rate, lr)

n_epochs = 15
n_iterations = math.ceil(len(X_train_scaled) / batch_size) * n_epochs
onecycle = OneCycleScheduler(n_iterations, max_lr=0.05)
history = model.fit(X_train_scaled, y_train, epochs=n_epochs, batch_size=batch_size,
                    validation_data=(X_valid_scaled, y_valid),
                    callbacks=[onecycle])

Epoch 1/15
352/352 [==============================] - 4s 9ms/step - loss: 2.0598 - accuracy: 0.2882 - val_loss: 1.7692 - val_accuracy: 0.3772
Epoch 2/15
352/352 [==============================] - 3s 8ms/step - loss: 1.7708 - accuracy: 0.3764 - val_loss: 1.6568 - val_accuracy: 0.4144
Epoch 3/15
352/352 [==============================] - 3s 8ms/step - loss: 1.6304 - accuracy: 0.4235 - val_loss: 1.6318 - val_accuracy: 0.4196
Epoch 4/15
352/352 [==============================] - 3s 8ms/step - loss: 1.5534 - accuracy: 0.4508 - val_loss: 1.6176 - val_accuracy: 0.4348
Epoch 5/15
352/352 [==============================] - 3s 8ms/step - loss: 1.4996 - accuracy: 0.4650 - val_loss: 1.6485 - val_accuracy: 0.4396
Epoch 6/15
352/352 [==============================] - 3s 9ms/step - loss: 1.4580 - accuracy: 0.4834 - val_loss: 1.6065 - val_accuracy: 0.4384
Epoch 7/15
352/352 [==============================] - 3s 9ms/step - loss: 1.4209 - accuracy: 0.4968 - val_loss: 1.6135 - val_accuracy: 0.4434
Epoch 